In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy, scipy as sklearn, librosa, urllib
import librosa.display
from IPython.display import Audio
import json 
import seaborn as sns
from sklearn.cluster import KMeans
import csv
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import keras
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from os import path

from itertools import cycle
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc, silhouette_score,roc_auc_score, precision_recall_fscore_support
from tqdm import tqdm
import opensmile


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

from tensorflow.keras import regularizers
from joblib import dump, load
from sklearn.metrics import mean_squared_error
from tensorflow.keras.datasets import mnist

from keras.models import load_model

import random



In [2]:
def get_emo_df(emotion, dataframe):
    emo = []
    tot = 0
    print(f"Processing {emotion} ...")
    for id, name in zip(dataframe["filename"],dataframe["label"]):
        if f"{emotion}" in name:
            emo.append(id)
            tot += 1
    print(f"Processed {tot} file for {emotion}")


    mask = dataframe['filename'].isin(emo)
    emo_df = dataframe[mask].reset_index()
    emo_df = emo_df.drop(['index'], axis=1)
    return emo_df


In [8]:
def get_data(df,seed):
    X = pd.concat(df["features"].tolist(), ignore_index=True)
    _X= (X-X.min())/(X.max()-X.min())
    


    X_train, X_test = train_test_split(_X, test_size=.2, random_state=seed)
    X_train, X_val = train_test_split(X_train, test_size=.1, random_state=seed)
    # print(len(X_train), len(X_val), len(X_test))
    return X_train, X_val, X_test

In [6]:
# clear_df = pd.read_pickle("../utils/data/clear_functional.pkl")
seed = 99
clear_df = pd.read_pickle("../utils/data/clear_functional_emovo.pkl")
seed = 23

# clear_df = pd.read_pickle("../utils/data/pitch_functional_demos.pkl")
# clear_df = pd.read_pickle("../utils/data/pitch_functional_emovo.pkl")
labels = clear_df["label"].unique()
labels

array(['disgust', 'joy', 'neutrality', 'fear', 'anger', 'surprise',
       'sadness'], dtype=object)

In [7]:
df = []

for i,emotion in enumerate(labels):
    df.append(get_emo_df(emotion,clear_df))

Processing disgust ...
Processed 84 file for disgust
Processing joy ...
Processed 84 file for joy
Processing neutrality ...
Processed 84 file for neutrality
Processing fear ...
Processed 84 file for fear
Processing anger ...
Processed 84 file for anger
Processing surprise ...
Processed 84 file for surprise
Processing sadness ...
Processed 83 file for sadness


In [9]:
train = []
val = []
test = []
print(seed)
for data in df:
    tr, v, te = get_data(data, seed)
    train.append(tr)
    val.append(v)
    test.append(te)

23


In [10]:
def get_rec_error(ae, seed, flag):
    errors = {}
    for i in labels:
        errors[i] = None
    for i in range(len(labels)):
        data = df[i]
        _X_train, _X_val, _X_test = get_data(data, seed)
        if flag:
            _X_test = _X_train
        reconstructed_X = ae.predict(_X_test)
        errors[labels[i]] = mean_squared_error(_X_test,reconstructed_X)
        # mse_per_feature = ((_X_test - reconstructed_X)**2).mean(axis=0)
        # errors[labels[i]] = np.mean(mse_per_feature)
    return errors
        

In [52]:
def load_models(model_path,labels, i ):
    model = (model_path + labels[i] + ".keras")
    print(model)
    return load_model(model)

In [49]:
EMOVO_model_path = "../utils/paper/models/EMOVO/ae_"
DEMoS_model_path = "utils/paper/models/DEMoS/ae_"

In [53]:
autoencoders = []
for i,emotion in enumerate(labels):
    autoencoders.append(load_models(EMOVO_model_path, labels, i))

../utils/paper/models/EMOVO/ae_disgust.keras
../utils/paper/models/EMOVO/ae_joy.keras
../utils/paper/models/EMOVO/ae_neutrality.keras
../utils/paper/models/EMOVO/ae_fear.keras
../utils/paper/models/EMOVO/ae_anger.keras
../utils/paper/models/EMOVO/ae_surprise.keras
../utils/paper/models/EMOVO/ae_sadness.keras
